<a href="https://colab.research.google.com/github/Martinmbiro/XO-binary-classification/blob/main/01%20XO%20modular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Implementing Modules**
> In this notebook, I'll be creating modules for the most reusable code from one of my previous GitHub repositories: [`Pytorch computer vision basics`](https://github.com/Martinmbiro/Pytorch-computer-vision-basics/blob/main/04%20Implementing%20TinyVGG%20model%20architecture.ipynb)

> 💎 **Pro Tip**
+ Modules help organize code logically, promote code reusability and cleaner code

> 📝 **Note**  
+ All the steps we followed in the notebook linked above remain the same, except the neural network I'll be using here (since it's a binary classification problem)
+ To **write** a code cell's content into a `*.py`, file we'll use the _magic command_ `%%writefile filename.py`
+ To **append** a code cell's content into a `*.py`, file we'll use the _magic command_ `%%writefile -a filename.py`

In [ ]:
import torch, torchvision

print(f'torch version: {torch.__version__}')
print(f'torchvison version: {torchvision.__version__}')

torch version: 2.6.0+cu124
torchvison version: 0.21.0+cu124


### Load the data
> First, we'll create a directory to hold all the custom modules we write
+ To create directories, we'll make use of the [`pathlib`](https://docs.python.org/3/library/pathlib.html) python module

In [ ]:
from pathlib import Path

modules_dir = Path('helper_modules')
modules_dir.mkdir(parents=True, exist_ok=True)

> Next, we'll define a function that filters `X` and `O` images from the [`EMNIST`](https://pytorch.org/vision/main/generated/torchvision.datasets.EMNIST.html) dataset. The function will return:
+ `train_dl` - A `Dataloader` for training `Dataset`
+ `test_dl` - A `Dataloader` for test `Dataset`
+ `y_true` - A `ndarray` of true class labels from test `Dataset`
+ `label_map` - A `dict` mapping class indices to class names

In [ ]:
%%writefile helper_modules/data_loader.py
"""
Contains functionality for creating DataLoaders from EMNIST dataset
"""
import torch, numpy as np, os
import torchvision.transforms as T
from torch.utils.data import Subset, DataLoader, Dataset
from torchvision import datasets

# batch size and num_workers
BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

def create_dataloaders() -> tuple[DataLoader, DataLoader, np.ndarray, dict]:
  """Creates training and testing DataLoaders of the EMNIST dataset (for X and O only).
  Also, returns y_true as well as a dictionary mapping indices to labels (for X and 0)

  Returns
  -------
    train_dl: torch.utils.data.Dataloader
          Training DataLoader

    test_dl: torch.utils.data.Dataloader
          Testing DataLoader

    y_true: np.ndarray
          An ndarray of true labels from the test Subset that was used to create testing DataLoader

    label_map: dict
          A dictionary mapping index to classes
  """
  # class to warp around a Subset
  class set_wrapper(Dataset):
    def __init__(self, subset:Subset):
      self.subset = subset

    def __len__(self):
      return len(self.subset)

    def __getitem__(self, index):
      # get image and label at specified index
      x, y = self.subset[index]
      # transform label 15 -> 0 and 24 -> 1
      y = 0 if y==15 else 1
      return x, y

  # get the original dataset
  train_data = datasets.EMNIST(
      root='data', download=True, train=True, split='letters', transform=T.ToTensor())
  test_data = datasets.EMNIST(
      root='data', download=True, train=False, split='letters', transform=T.ToTensor())
  # label_map
  label_map = {0:'O', 1:'X'}

  # get indices for train and test data (where target==24 | target==15)
  train_indices = np.where((train_data.targets == 24) | (train_data.targets == 15))[0]
  test_indices = np.where((test_data.targets == 24) | (test_data.targets == 15))[0]
  np.random.shuffle(test_indices) # shuffle test_indices

  # from indices gotten above, create subsets for train and test
  train_set = Subset(dataset=train_data, indices=train_indices)
  test_set = Subset(dataset=test_data, indices=test_indices)

  # from the subsets above, wrap in custom set_wrapper class
  train_dataset, test_dataset = set_wrapper(train_set), set_wrapper(test_set)

  # get the targets from test Subset
  y_true = list()
  for x in range(len(test_set)):
    _, y = test_set[x]
    y_true.append(y)
  # turn targets (15 and 24 into 0 and 1 respectively)
  y_true = [0 if x==15 else 1 for x in y_true]
  # turn the targets list into a numpy array
  y_true = np.array(y_true)

  # create train and test DataLoaders from the Subsets created above
  train_dl = DataLoader(
    dataset=train_dataset, batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS, pin_memory=True, shuffle=True)

  test_dl = DataLoader(
    dataset=test_dataset, batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS, pin_memory=True, shuffle=False)

  # return
  return train_dl, test_dl, y_true, label_map

Writing helper_modules/data_loader.py


## Build the [`TinyVGG`](https://www.youtube.com/watch?v=HnWIHWFbuUQ) architecture
> Basically, the original `TinyVGG` network consists of two _convolutional_ blocks and a _classifier_ block
+ Each _convolutional_ block consists of two [`nn.Conv2d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d) with `10` `out_channels`, two [`nn.ReLU`](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html#torch.nn.ReLU) activations for non-linearity and a [`nn.MaxPool2d`](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html#torch.nn.MaxPool2d) layer
+ The _classifier_ block consists of one [`nn.Flatten`](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html#torch.nn.Flatten) and a [`nn.Linear`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear) for classification

> The function defined here will return a `model`, `Optimizer` and `loss function`


> 📝 **Note**

> I'll be making a few modification to the original `TinyVGG` architecture as follows:
+ The number of `out_channels` begins at `64` for the first `nn.Conv2d` layer and doubles for each subsequent `nn.Conv2d` layer
+ An [`nn.BatchNorm2d`](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html#torch.nn.BatchNorm2d) layer after each `nn.Conv2d` layer, but before the `nn.ReLU` non-linear activation
+ An [`nn.BatchNorm1d`](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html#torch.nn.BatchNorm1d) layer after the `nn.Flatten` and first `nn.Linear` layer in the `classifier` block
+ An [`nn.Dropout`](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html#torch.nn.Dropout) layer just before the output layer of the `classifier` block
+ Also, we'll use [`torch.optim.SGD`](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD) as optimizer and [`torch.nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#crossentropyloss) as loss function, since this is a multi-class classification problem


In [ ]:
%%writefile helper_modules/model_builder.py
import torch
from torch import nn
from torch.optim import SGD

def get_model(device:str) -> tuple[nn.Module, torch.optim.Optimizer, nn.Module]:
  """A function that returns a model, optimizer and loss function

  Parameters
  -------
    device: str
        The device on which to perform computation

  Returns
  -------
    model: torch.nn.Module
        A TinyVGG architecture model

    opt: torch.Optimizer
        An optimizer

    loss_fn: torch.nn.Module
        A loss function for multi-class classification
  """
  torch.manual_seed(42)
  torch.cuda.manual_seed(42)
  # define model
  class TinyVGG(nn.Module):
    def __init__(self):
      super().__init__()
      # conv_block1
      self.conv_b1 = nn.Sequential(
          nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          nn.Conv2d(64, 128, 3, padding=1),
          nn.BatchNorm2d(128),
          #nn.Dropout(p=0.15),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2))

      # conv_block2
      self.conv_b2 = nn.Sequential(
          nn.Conv2d(128, 256, 3, padding=1),
          nn.BatchNorm2d(256),
          nn.ReLU(),
          nn.Conv2d(256, 512, 3, padding=1),
          nn.BatchNorm2d(512),
          # nn.Dropout(p=0.15),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2))

      # classifier
      self.classifier = nn.Sequential(
          nn.Flatten(),
          nn.BatchNorm1d(25088),
          nn.Linear(in_features=25088, out_features=256),
          nn.BatchNorm1d(256),
          nn.Dropout(p=0.05), # droput regularization
          nn.Linear(256, 2))

    def forward(self, x):
      return self.classifier(self.conv_b2(self.conv_b1(x)))

  # get an object of the model
  model = TinyVGG().to(device)

  # optimizer
  '''opt = torch.optim.Adam(params=model.parameters(),
                         lr=0.0009) # learning rate'''
  # optimizer
  opt = torch.optim.SGD(
      params=model.parameters(),
      lr=0.0001,  # learning rate
      momentum=0.5) #

  # loss_function
  loss_fn = nn.CrossEntropyLoss()

  return model, opt, loss_fn

Writing helper_modules/model_builder.py


## Early stopping
> 💎 **Pro Tip**

> [Early stopping](https://www.linkedin.com/advice/1/what-benefits-drawbacks-early-stopping#:~:text=Early%20stopping%20is%20a%20form,to%20increase%20or%20stops%20improving.) is a mechanism of stopping training when the validation loss stops improving; with a view to preventing _overfitting_ on the training data
+ Here, we'll create a class to take care of _early-stopping_

In [ ]:
%%writefile helper_modules/utils.py
"""
contains helper functions for tasks like loading & saving models,
earlystopping, plotting training metrics, e.t.c
"""
import torch, pathlib, numpy as np, matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import ConfusionMatrixDisplay
from torch.utils.data import Dataset, Subset
from pathlib import Path
from copy import deepcopy

# declarea class to implement earlystopping
class EarlyStopping:
  """A class that implements EarlyStopping mechanism

  Parameters
  -------
    score_type:
      'metric' for a metric score (eg. f1_score) and 'loss' for
        a loss function (eg. CrossEntropyLoss)

    min_delta: float
      How much of a difference in loss is to be considered worthy to continue training

    patience: int
      The number of epochs to wait after the last improvement before stopping
  """
  def __init__(self, score_type:str, min_delta:float=0.0, patience:int=5): # constructor
    self.counter = 0
    self.patience = patience
    self.min_delta = min_delta
    self.score_type = score_type
    self.best_epoch = None
    self.best_score = None
    self.best_state_dict = None
    self.stop_early = False

    if (self.score_type != 'metric') and (self.score_type != 'loss'):
      err_msg = 'score_type can only be "metric" or "loss"'
      raise Exception(err_msg)

  def __call__(self, model:torch.nn.Module, ep:int, ts_score:float):
    """Pass the following arguments to the object name of EarlyStopping to call this method

    Parameters
    -------
      model: torch.nn.Module
          The object name of the model being trained (subclasses nn.Module)

      ep: int
          The current epoch in the training / optimization loop

      ts_score: float
          The score (loss or metric) being used to decide early stopping mechanism
    """
    if self.best_epoch is None: # for first time:
      self.best_epoch = ep # store current epoch
      self.best_score = ts_score # store current loss as best loss
      # make a copy of current model's state_dict
      self.best_state_dict = deepcopy(model.state_dict())

    # if previous loss - current loss exceeds min_delta: (for loss function)
    elif (self.best_score - ts_score >= self.min_delta) and (self.score_type == 'loss'):
      self.best_epoch = ep # store current epoch
      self.best_score = ts_score # store current loss as best
      # make a copy of current model's state_dict
      self.best_state_dict = deepcopy(model.state_dict())
      self.counter = 0 # restore counter to zero

    # if current metric - previous. metric exceeds min_delta: (for metric)
    elif (ts_score - self.best_score >= self.min_delta) and (self.score_type == 'metric'):
      self.best_epoch = ep # store current epoch
      self.best_score = ts_score # store current loss as best
      # make a copy of current model's state_dict
      self.best_state_dict = deepcopy(model.state_dict())
      self.counter = 0 # restore counter to zero

    else: # otherwise
      self.counter += 1 # increment counter each time
      if self.counter >= self.patience:
        self.stop_early = True

Writing helper_modules/utils.py


## Model training / evaluation
> Here, I'll define functions for training and testing batches of data, as well as a function to return prediction labels, `y_pred` and prediction probabilities `y_proba`

In [ ]:
%%writefile helper_modules/train_test.py
import torch, numpy as np
import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score

# del torch, F, f1_score, accuracy_score

# function for model training
def train_batches(model:torch.nn.Module, train_dl:torch.utils.data.DataLoader,
                optimizer:torch.optim.Optimizer, loss_fn:torch.nn.Module, device:str) -> tuple[float, float, float]:
  """Trains model on all batches of train-set DataLoader and returns
      average training loss, accuracy and f1_score

  Parameters
  -------
    model: torch.nn.Module
        The model being trained

    train_dl: torch.utils.data.DataLoader
        DataLoader for training data

    optimizer: torch.optim.Optimizer
        The optimizer

    loss_fn: torch.nn.Module
        Function used to calculate loss

    device: str
        The device on which computation occurs

  Returns
  -------
    ls: float
        average test loss across all batches of data
    acc: float
        average test accuracy across all batches of data
    f1: float
        average test f1_score across all batches of data
  """
  # for reproducability
  torch.manual_seed(0)
  torch.cuda.manual_seed(0)
  ls, acc, f1 = 0, 0, 0

  #training mode
  model.train()

  for x, y in train_dl:
    # move x, y to device
    x, y = x.to(device), y.to(device)
    # zero_grad
    optimizer.zero_grad()

    # forward pass
    logits = model(x)
    y_pred = F.softmax(logits, dim=1).argmax(dim=1).cpu().numpy()

    # loss
    loss = loss_fn(logits, y)
    # accumulate values
    ls += loss.item()
    acc += accuracy_score(y_true=y.cpu().numpy(), y_pred=y_pred)
    f1 += f1_score(y_true=y.cpu().numpy(), y_pred=y_pred)

    # back propagation
    loss.backward()
    # optmizer step
    optimizer.step()

  # compute averages
  ls /= len(train_dl)
  acc /= len(train_dl)
  f1 /= len(train_dl)

  # return values
  return ls, acc, f1

# function for model testing
def test_batches(model:torch.nn.Module, test_dl:torch.utils.data.DataLoader,
                loss_fn:torch.nn.Module, device:str) -> tuple[float, float, float]:
  """Evaluates model on all batches of test-set DataLoader and returns
      average test loss, accuracy and f1_score

  Parameters
  -------
    model: torch.nn.Module
        The model being evaluated

    test_dl: torch.utils.data.DataLoader
        DataLoader for test data

    loss_fn: torch.nn.Module
        Function used to calculate loss

    device: str
        The device on which computation occurs

  Returns
  -------
    ls: float
        average test loss across all batches of data
    acc: float
        average test accuracy across all batches of data
    f1: float
        average test f1_score across all batches of data
  """
  ls, f1, acc = 0, 0, 0

  # evaluation-mode
  model.eval()

  with torch.inference_mode():
    for x, y in test_dl:
      # move x, y to device
      x, y = x.to(device), y.to(device)

      # forward pass
      logits = model(x)
      y_pred = F.softmax(logits, dim=1).argmax(dim=1).cpu().numpy()

      # loss
      loss = loss_fn(logits, y)

      # accumulate values
      ls += loss.item()
      acc += accuracy_score(y_true=y.cpu().numpy(), y_pred=y_pred)
      f1 += f1_score(y_true=y.cpu().numpy(), y_pred=y_pred)

  # compute averages
  ls /= len(test_dl)
  acc /= len(test_dl)
  f1 /= len(test_dl)

  # return values
  return ls, acc, f1

# function to return prediction labels (y_pred) and prediction probabilities (y_proba)
def get_preds_proba(model:torch.nn.Module, test_dl:torch.utils.data.DataLoader,
                    device:str) -> tuple[np.ndarray, np.ndarray]:
  """A function that returns y_pred and y_proba from the passed DataLoader

  Parameters
  -------
    model: torch.nn.Module
        A neural network that subclasses torch.nn.Module

    test_dl: torch.utils.data.DataLoader
        A DataLoader for the test dataset

  Returns
  -------
    y_pred: np.ndarray
        A numpy ndarray with prediction labels

    y_proba: np.ndarray
        A numpy ndarray with prediction probabilities
  """
  # empty lists
  y_preds, y_proba = list(), list()
  with torch.inference_mode():
    model.eval() # set eval mode
    for x, _ in test_dl:
      # move x to device
      x = x.to(device)

      # make prediction
      logits = model(x)

      # prediction and probabilites
      proba = F.softmax(logits, dim=1)
      pred = F.softmax(logits, dim=1).argmax(dim=1)

      # append
      y_preds.append(pred)
      y_proba.append(proba)

  y_preds = torch.concatenate(y_preds).cpu().numpy()
  y_proba = torch.concatenate(y_proba).cpu().numpy()

  return y_preds, y_proba

Writing helper_modules/train_test.py


## Plot results
> Here, I'll define helper functions for plotting training metrics

In [ ]:
%%writefile -a helper_modules/utils.py

# function to plot train and test results
def plot_train_results(ep_list:list, train_score:list, test_score:list,
                       ylabel:str, title:str, best_epoch:int):
  """A function that plots train and test results against each other

  Parameters
  -------
    ep_list: list
      A list containing all epochs used in the optimization loop

    train_score: list
      A list containing a specific training score from the optimization loop

    test_score: list
      A list containing a specific training score from the optimization loop

    y_label: str
      y-axis label for the plot

    title: str
      Title for the plot

    best_epoch: int
      Best epoch for which early stopping occurred
  """
  f, ax = plt.subplots(figsize=(5, 3), layout='constrained')

  # train loss
  ax.plot(ep_list, train_score, label='Training',
          linewidth=1.7, color='#0047ab')

  # test loss
  ax.plot(ep_list, test_score, label='Validation',
          linewidth=1.7, color='#990000')
  # vertical line (for early stopping)
  if best_epoch is not None:
    ax.axvline(best_epoch, linestyle='--', color='#000000', linewidth=1.0,
             label=f'Best ep ({best_epoch})')

  # axis, title
  ax.set_title(title, weight='black')
  ax.set_ylabel(ylabel)
  ax.set_xlabel('Epoch')
  ax.tick_params(axis='both', labelsize=9)
  plt.grid(color='#e5e4e2')

  # legend
  f.legend(fontsize=9, loc='upper right',
          bbox_to_anchor=(1.28, 0.93),
          fancybox=False)

  plt.show()

# function to plot confusion matrix
def plot_confusion_matrix(y_true:np.ndarray, y_pred:np.ndarray):
  """A function that plots Confusion Matrix for all classes

  Parameters
  -------
    y_true: np.ndarray
      An ndarray containing true label values

    y_pred: np.ndarray
      An ndarray containing predicted label values
  """
  # define figure and plot
  _, ax = plt.subplots(figsize=(3.0,3.0), layout='compressed')
  # plot
  ConfusionMatrixDisplay.from_predictions(
      y_true=y_true,
      y_pred=y_pred, cmap='Blues', colorbar=False, ax=ax)

  # set x and y labels
  ax.set_ylabel('True Labels', weight='black')
  ax.set_xlabel('Predicted Labels', weight='black',
                  color='#dc143c')
  # set tick size and position
  ax.xaxis.tick_top()
  ax.xaxis.set_label_position('top')
  ax.tick_params(axis='both', labelsize=9)

  # change annotation font
  for txt in ax.texts:
    txt.set_fontsize(9)

  plt.show()

Appending to helper_modules/utils.py


## Save model
> 🔔 **Info**

> Pytorch's recommended way of saving a model is by saving its [`state_dict`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.state_dict). To do this, the [documentation](https://pytorch.org/tutorials/beginner/saving_loading_models.html#save-load-state-dict-recommended) recommends calling [`torch.save(obj=model.state_dict(), f=PATH)`](https://pytorch.org/docs/stable/generated/torch.save.html#torch-save)
+ `f` - a file-like object or a string or `os.PathLike` object containing a file name. To work with paths, we'll use Python's [`pathlib`](https://docs.python.org/3/library/pathlib.html) module
+ A common PyTorch convention is to save models using either a `.pt` or `.pth` file extension
+ Also, it's good practice to move the model to the `cpu` before saving its `state_dict`

In [ ]:
%%writefile -a helper_modules/utils.py

# function to save model to specified directory
def save_model(model:torch.nn.Module, path:pathlib.PosixPath):
  """Function to save model to a specified path

  Parameters
  -------
    model: torch.nn.Module
      The model to save

    path: pathlib.PosixPath
      Path to save model's state_dict
  """
  torch.save(obj=model.cpu().state_dict(), f=path)
  print(f"MODEL'S state_dict SAVED TO: {path}")

Appending to helper_modules/utils.py


### Load saved model
> To load a previously saved model's `state_dict`, we call
 [`torch.load(f=PATH, weights_only=True)`](https://pytorch.org/docs/stable/generated/torch.load.html#torch.load) that loads an object saved using [`torch.save()`](https://pytorch.org/docs/stable/generated/torch.save.html#torch-save) from a file:

```
    model = TheModelClass(*args, **kwargs)
    model.load_state_dict(torch.load(PATH, weights_only=True))
    model.eval()
```

> 🔔 **Info**
+ Remember that you must call `model.eval()` before running inference
+ `f` - a file-like object or a string or `os.PathLike` object containing a file name. To work with paths, we'll use Python's [`pathlib`](https://docs.python.org/3/library/pathlib.html) module
+ Note that a `model` class must have been defined earlier, before calling [`model.load_state_dict()`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.load_state_dict) on the object

In [ ]:
%%writefile -a helper_modules/utils.py

# function to load model from a specified path
def load_model(model:torch.nn.Module, path:pathlib.PosixPath):
  """Function to load model from a specified path

  Parameters
  -------
    model:torch.nn.Module
        A new object of the model class

    path:pathlib.PosixPath
        Path pointing to a previously saved model's state_dict

  Return
  -------
    model:torch.nn.Module
      model returned after loading state_dict
  """
  # overwrite stat_dict
  model.load_state_dict(
      torch.load(f=path, weights_only=True))

Appending to helper_modules/utils.py


## Make inference
> Here, I'll declare functions to make inference:
+ On a single random image
+ On multiple `(12)` random images

In [ ]:
%%writefile -a helper_modules/utils.py

# function to make inference on a single random image
def make_single_inference(model:torch.nn.Module, dataset:torch.utils.data.Dataset,
                          label_map:dict, device:str):
  """Makes inference using a random data point from the test dataset

  Parameters
  -------
    model: torch.nn.Module
      A model (subclassing torch.nn.Module) to make inference

    dataset: torch.utils.data.Dataset
      The Dataset to use for testing purposes

    label_map: dict
      A dictionary maping indices to labels (eg. {0:'O', 1:'X'})

    device: str
      Device on which to perform computation
  """
  # get random image from test_set
  idx = np.random.choice(len(dataset))
  img, lb = dataset[idx]

  # make prediction
  with torch.inference_mode():
    model.to(device) # move model to device
    model.eval() # set eval mode
    lgts = model.to(device)(img.unsqueeze(0).to(device))
    pred = F.softmax(lgts, dim=1).argmax(dim=1)

  # print actual retrieved image
  plt.figure(figsize=(1.0, 1.0))
  # title with label
  if pred==lb:
    plt.title(
        f'Actual: {label_map[lb]}\nPred: {label_map[pred.item()]}',
        fontsize=8)
  else: # if labels do not match, title = with red colour
    plt.title(
        f'Actual: {label_map[lb]}\nPred: {label_map[pred.item()]}',
        fontsize=8, color='#de3163', weight='black')
  plt.axis(False)
  plt.imshow(img.squeeze(), cmap='gray')
  plt.show()

# function to make inference on multiple random images
def make_multiple_inference(model:torch.nn.Module, dataset:torch.utils.data.Dataset,
                            label_map:dict, device:str):
  """Makes inference using a random data point from the test dataset

  Parameters
  -------
    model: torch.nn.Module
      A model (subclassing torch.nn.Module) to make inference

    dataset: torch.utils.data.Dataset
      The Dataset used for evaluation purposes

    label_map: dict
      A dictionary maping indices to labels (eg. {0:'O', 1:'X'})

    device: str
      Device on which to perform computation
  """
  # get array of 12 random indices of images in test_dataset
  indices = np.random.choice(len(dataset),
                                  size= 12, replace=False)
  # create subset from the 12 indices
  sub_set = Subset(dataset=dataset, indices=indices)

  # define a figure and subplots
  f, axs = plt.subplots(2, 6, figsize=(6,5), layout='compressed')

  # move model to device & set eval mode
  model.to(device)
  model.eval()

  # loop through each subplot
  for i, ax in enumerate(axs.flat):
    img, lb = sub_set[i] # return image and label

    # make inference on image retuned
    with torch.inference_mode():
      lg = model(img.unsqueeze(0).to(device))
      pred = F.softmax(lg, dim=1).argmax(dim=1)

    ax.imshow(img.squeeze(), cmap='gray')
    ax.axis(False)
    if pred==lb:
      ax.set_title(
          f'Actual: {label_map[lb]}\nPred: {label_map[pred.item()]}',
          fontsize=8)
    else: # if labels do not match, title = with red colour
      ax.set_title(
          f'Actual: {label_map[lb]}\nPred: {label_map[pred.item()]}',
          fontsize=8, color='#de3163', weight='black')

  f.suptitle('Inference Made on 12 Random Test Images',
            weight='black',
            y=0.83)
  plt.show()

Appending to helper_modules/utils.py


## Archive modules
> Here, we'll create a function to archive all the modules `*.py` files into a `*.zip` file with the help of the [`zipfile`](https://docs.python.org/3/library/zipfile.html) python module

> ✋ **Info**
+ The `zip` file containing the helper modules will be then uploaded to the GitHub repository [here](https://github.com/Martinmbiro/XO-binary-classification/tree/main/helper%20modules). That way, the modules can be downloaded and extracted dynamically in code

In [ ]:
import zipfile, pathlib

def archive_modules(path_to_files:pathlib.PosixPath, zip_name:str):
  """Takes a path with files and archives .py files in that path
  Parameters
  -------
    path_to_files: path_to_files:pathlib.PosixPath
      path to the directory in question
    zip_name: str
      Name to give the zipfolder
  """
  with zipfile.ZipFile(file=zip_name, mode='w', compression=zipfile.ZIP_STORED) as zipf:
    for file_path in path_to_files.glob('*.py'):
        zipf.write(filename=file_path, arcname=file_path.name)

In [ ]:
%%time
# archive modules
archive_modules(modules_dir, 'modules.zip')

CPU times: user 1.85 ms, sys: 0 ns, total: 1.85 ms
Wall time: 2.14 ms


> ▶️ **Up Next**
+ Having created modules out of the most reusable code, I'll implement an end to tend project for classifying `X` and `O` images in the subsequent notebook, `01. XO Culimination.ipynb`